In [102]:
from copy import copy
from datetime import datetime

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tpot import TPOTClassifier

In [77]:
df = pd.read_csv("../dataset.csv")
df.head()

,EntryId,NodeId,AbsoluteTime,RelativeTime,Datetime,Temperature,MeanTempDay,HeatIndex,RelativeHumidity,LightSensorOneWaveLength,...,ActivityOfOccupants,StateOfDoor,StateOfWindow,acceptability_80,acceptability_90,tmp_cmf,tmp_cmf_80_low,tmp_cmf_80_up,tmp_cmf_90_low,tmp_cmf_90_up
0,1,1,1458031648545,1,15/03/2016,20.48,8.6,20.587526,42.332,185.71,...,0,0,0,0,0,20.9,21.1,22.5,21.1,21.9
1,2,4,1458031648645,1,15/03/2016,20.73,8.6,20.674882,39.983,214.29,...,0,0,0,0,0,20.9,21.1,22.5,21.1,21.9
2,3,3,1458031648795,1,15/03/2016,20.48,8.6,20.504889,38.687,197.14,...,0,0,0,0,0,20.9,21.1,22.5,21.1,21.9
3,4,2,1458031651038,3,15/03/2016,20.89,8.6,20.783859,40.712,174.29,...,0,0,0,0,0,20.9,21.1,22.5,21.1,21.9
4,5,4,1458031652010,4,15/03/2016,20.73,8.6,20.674882,39.983,211.43,...,0,0,0,0,0,20.9,21.1,22.5,21.1,21.9


In [78]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 531242 entries, 0 to 531241
Data columns (total 22 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   EntryId                   531242 non-null  int64  
 1   NodeId                    531242 non-null  int64  
 2   AbsoluteTime              531242 non-null  int64  
 3   RelativeTime              531242 non-null  int64  
 4   Datetime                  531242 non-null  object 
 5   Temperature               531242 non-null  float64
 6   MeanTempDay               531242 non-null  float64
 7   HeatIndex                 531242 non-null  float64
 8   RelativeHumidity          531242 non-null  float64
 9   LightSensorOneWaveLength  531242 non-null  float64
 10  LightSensorTwoWaveLength  531242 non-null  float64
 11  NumberOfOccupants         531242 non-null  int64  
 12  ActivityOfOccupants       531242 non-null  int64  
 13  StateOfDoor               531242 non-null  i

In [79]:
df = shuffle(df)

In [80]:
# target_name = "acceptability_80"
# target_name = "acceptability_90"
# target_name = "tmp_cmf"
# target_name = "tmp_cmf_80_low"
# target_name = "tmp_cmf_80_up"
# target_name = "tmp_cmf_80_low"
# target_name = "tmp_cmf_90_up"
# target_name = "tmp_cmf_90_low"
target_name = "acceptability_90"

In [81]:
df.rename(columns={target_name: 'target'}, inplace=True)

df['AbsoluteTime'] = pd.to_datetime(df['AbsoluteTime'], unit='ms')

df['hour'] = df['AbsoluteTime'].dt.hour
df['minute'] = df['AbsoluteTime'].dt.minute
df['second'] = df['AbsoluteTime'].dt.second

In [82]:
df.columns

Index(['EntryId', 'NodeId', 'AbsoluteTime', 'RelativeTime', 'Datetime',
       'Temperature', 'MeanTempDay', 'HeatIndex', 'RelativeHumidity',
       'LightSensorOneWaveLength', 'LightSensorTwoWaveLength',
       'NumberOfOccupants', 'ActivityOfOccupants', 'StateOfDoor',
       'StateOfWindow', 'acceptability_80', 'target', 'tmp_cmf',
       'tmp_cmf_80_low', 'tmp_cmf_80_up', 'tmp_cmf_90_low', 'tmp_cmf_90_up',
       'hour', 'minute', 'second'],
      dtype='object')

In [105]:
features = [
    'Temperature',
    'MeanTempDay',
    'HeatIndex',
    'RelativeHumidity',
    'LightSensorOneWaveLength',
    'LightSensorTwoWaveLength',
    'NumberOfOccupants',
    'ActivityOfOccupants',
    'StateOfDoor',
    'hour',
    'minute',
    'second',
]

In [84]:
y = df['target']
X = df[features]

In [112]:
export_features = copy(features)
export_features.append('target')

df[export_features].to_csv("processed_data.csv")

In [59]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y.values, test_size=0.33)

In [35]:
pipeline_optimizer = TPOTClassifier(generations=5, n_jobs=-1, early_stop=3, verbosity=3)

In [46]:
pipeline_optimizer.fit(X_train, y_train)

Optimization Progress:   0%|          | 0/1100 [00:00<?, ?pipeline/s]



TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.


RuntimeError: A pipeline has not yet been optimized. Please call fit() first.

In [40]:
score = pipeline_optimizer.score(X_test, y_test)
score

0.9999942958188353

In [73]:
pipeline_optimizer.export(f"{score}_{datetime.today().strftime('%Y%m%d%H%M%S')}_tpot_exported_pipeline.py",
                          data_file_path="processed_data.csv")

RuntimeError: A pipeline has not yet been optimized. Please call fit() first.